# 5.1

In [1]:
import numpy as np
from scipy.stats import norm  

def monte_carlo_exp_integral(n=100, seed=None, alpha=0.05):
    rng = np.random.default_rng(seed)
    x = rng.uniform(0.0, 1.0, size=n)
    y = np.exp(x)

    mu_hat = y.mean()
    se = y.std(ddof=1) / np.sqrt(n)        
    z = norm.ppf(1 - alpha / 2)          
    ci = (mu_hat - z * se, mu_hat + z * se)
    return mu_hat, ci

if __name__ == "__main__":
    estimate, (lower, upper) = monte_carlo_exp_integral(n=100, seed=42)
    print(f"Monte Carlo estimate: {estimate:.6f}")
    print(f"95% confidence interval: ({lower:.6f}, {upper:.6f})")

Monte Carlo estimate: 1.687728
95% confidence interval: (1.598288, 1.777168)


# 5.2

In [2]:
import numpy as np
from scipy.stats import norm  

def antithetic_exp_integral(n_pairs=50, seed=None, alpha=0.05):
    rng = np.random.default_rng(seed)
    u = rng.uniform(0.0, 1.0, size=n_pairs)  
    y1 = np.exp(u)        
    y2 = np.exp(1.0 - u)     
    g  = 0.5 * (y1 + y2)    

    mu_hat = g.mean()
    se     = g.std(ddof=1) / np.sqrt(n_pairs)  
    z      = norm.ppf(1.0 - alpha / 2.0)
    ci     = (mu_hat - z * se, mu_hat + z * se)
    return mu_hat, ci


if __name__ == "__main__":
    estimate, (lo, hi) = antithetic_exp_integral(n_pairs=50, seed=42)
    print(f"Antithetic Monte Carlo estimate: {estimate:.6f}")
    print(f"95% confidence interval: ({lo:.6f}, {hi:.6f})")


Antithetic Monte Carlo estimate: 1.714106
95% confidence interval: (1.698623, 1.729589)


# 5.3

In [3]:
import numpy as np
from scipy.stats import norm  

def control_variate_exp_integral(n=100, seed=None, alpha=0.05):
    rng = np.random.default_rng(seed)
    u  = rng.uniform(0.0, 1.0, size=n)   
    y  = np.exp(u)              
    h  = u                            
    mu_h = 0.5                         

    cov_yh = np.cov(y, h, ddof=1)[0, 1]
    var_h  = h.var(ddof=1)
    beta_hat = cov_yh / var_h

    g = y - beta_hat * (h - mu_h)

    mu_hat = g.mean()
    se     = g.std(ddof=1) / np.sqrt(n)  
    z      = norm.ppf(1.0 - alpha / 2.0)
    ci     = (mu_hat - z * se, mu_hat + z * se)
    return mu_hat, ci, beta_hat


if __name__ == "__main__":
    estimate, (lo, hi), beta = control_variate_exp_integral(n=100, seed=42)
    print(f"Control-variate estimate: {estimate:.6f}")
    print(f"95% confidence interval: ({lo:.6f}, {hi:.6f})")
    print(f"Estimated β: {beta:.4f}")


Control-variate estimate: 1.709709
95% confidence interval: (1.698680, 1.720739)
Estimated β: 1.6550


# 5.4

In [4]:
import numpy as np
from scipy.stats import norm  

def stratified_exp_integral(n_total=100, n_strata=10, seed=None, alpha=0.05):
    if n_total % n_strata:
        raise ValueError("n_total must be divisible by n_strata for equal allocation.")

    rng = np.random.default_rng(seed)
    m = n_strata
    n_j = n_total // m        
    w = 1.0 / m                    

    mu_hat = 0.0                  
    var_est = 0.0                  

    for j in range(m):
        a, b = j / m, (j + 1) / m              
        u = rng.uniform(a, b, size=n_j)                 
        y = np.exp(u)                               
        mean_j = y.mean()
        var_j  = y.var(ddof=1)

        mu_hat += w * mean_j
        var_est += w**2 * var_j / n_j

    se = np.sqrt(var_est)
    z  = norm.ppf(1.0 - alpha / 2.0)
    ci = (mu_hat - z * se, mu_hat + z * se)
    return mu_hat, ci


if __name__ == "__main__":
    estimate, (lo, hi) = stratified_exp_integral(n_total=100, n_strata=10, seed=42)
    print(f"Stratified estimate: {estimate:.6f}")
    print(f"95% confidence interval: ({lo:.6f}, {hi:.6f})")


Stratified estimate: 1.713285
95% confidence interval: (1.703711, 1.722859)


# 5.5

In [5]:
import numpy as np, heapq
from scipy import stats

def simulate_blocking_cv(
    N_customers: int,
    m_servers: int,
    rng: np.random.Generator,
    arrival_sampler,        
    service_sampler         
):
    inter = arrival_sampler(rng, size=N_customers)
    arrivals = np.cumsum(inter)

    busy_pq = []         
    Y = np.empty(N_customers, dtype=int)   

    for k, t in enumerate(arrivals):
        while busy_pq and busy_pq[0] <= t:    
            heapq.heappop(busy_pq)
        if len(busy_pq) < m_servers:
            s = service_sampler(rng)          
            heapq.heappush(busy_pq, t + s)
            Y[k] = 0
        else:
            Y[k] = 1                        
    return Y, inter

def cv_estimate(Y, inter, mu_h=1.0):
    Z = inter - mu_h
    beta_hat = np.cov(Y, Z, ddof=1)[0, 1] / np.var(Z, ddof=1)
    g = Y - beta_hat * Z
    theta_hat = g.mean()
    var_hat   = g.var(ddof=1) / len(g)       
    return theta_hat, var_hat, beta_hat

def exp_arrivals(rng, size, rate=1.0):
    return rng.exponential(1 / rate, size=size)

def exp_service(rng, mean=8.0):
    return rng.exponential(mean)

def poisson_exp_control_variates(
    reps=10,             
    N_customers=10_000,
    m_servers=10,
    seed0=42
):
    rng_master = np.random.default_rng(seed0)
    est_cv  = []     
    beta    = []        
    for r in range(reps):
        rng = np.random.default_rng(rng_master.integers(1 << 63))
        Y, inter = simulate_blocking_cv(
            N_customers, m_servers, rng,
            arrival_sampler=lambda R, size: exp_arrivals(R, size, rate=1.0),
            service_sampler=lambda R: exp_service(R, mean=8.0)
        )
        theta_hat, var_hat, beta_hat = cv_estimate(Y, inter)
        est_cv.append(theta_hat)
        beta.append(beta_hat)

    est_cv  = np.array(est_cv)
    beta    = np.array(beta)

    mean_hat = est_cv.mean()
    half_width = stats.t.ppf(0.975, df=reps-1) * est_cv.std(ddof=1) / np.sqrt(reps)
    ci = (mean_hat - half_width, mean_hat + half_width)

    print("Poisson arrivals (control variate):")
    print(f"  Mean blocked fraction = {mean_hat:.5f}")
    print(f"  95% CI = [{ci[0]:.5f}, {ci[1]:.5f}]")
    print(f"  β̂ (replications): {beta.mean():.4f} (avg)")

    return est_cv, beta, ci


if __name__ == "__main__":
    poisson_exp_control_variates()


Poisson arrivals (control variate):
  Mean blocked fraction = 0.11865
  95% CI = [0.11595, 0.12135]
  β̂ (replications): -0.0654 (avg)


# 5.6

In [8]:
import numpy as np, heapq
from scipy import stats

def simulate_blocking_given_interarrivals(inter, m_servers, service_sampler, rng):
    arrivals = np.cumsum(inter)
    busy = []                  
    blocked = 0

    for t in arrivals:
        while busy and busy[0] <= t:    
            heapq.heappop(busy)
        if len(busy) < m_servers:    
            s = service_sampler(rng)
            heapq.heappush(busy, t + s)
        else:                     
            blocked += 1
    return blocked / len(inter)

def exp_service(rng, mean=8.0):
    return rng.exponential(mean)

def compare_crn_vs_independent(
    reps=10, N_customers=10_000, m_servers=10,
    mean_service=8.0, seed0=12345
):
    p1, λ1, λ2 = 0.8, 0.8333, 5.0          
    rng_master = np.random.default_rng(seed0)

    diff_crn, diff_ind = [], []

    for r in range(reps):
        rng = np.random.default_rng(rng_master.integers(1 << 63))

        U  = rng.random(N_customers)  
        Uc = rng.random(N_customers)      

        inter_poiss = -np.log(U)        
        rate        = np.where(Uc < p1, λ1, λ2)
        inter_hyp   = -np.log(U) / rate     

        rng_s1 = np.random.default_rng(rng.integers(1 << 63))
        rng_s2 = np.random.default_rng(rng.integers(1 << 63))

        frac_poiss_crn = simulate_blocking_given_interarrivals(
            inter_poiss, m_servers,
            lambda R: exp_service(R, mean_service), rng_s1)

        frac_hyp_crn = simulate_blocking_given_interarrivals(
            inter_hyp, m_servers,
            lambda R: exp_service(R, mean_service), rng_s2)

        diff_crn.append(frac_hyp_crn - frac_poiss_crn)

        rngA = np.random.default_rng(rng.integers(1 << 63))
        rngB = np.random.default_rng(rng.integers(1 << 63))

        inter_poiss_ind = rngA.exponential(1.0, N_customers)

        Usel = rngB.random(N_customers)
        Uexp = rngB.random(N_customers)
        rate_ind = np.where(Usel < p1, λ1, λ2)
        inter_hyp_ind = -np.log(Uexp) / rate_ind

        rng_s3 = np.random.default_rng(rng.integers(1 << 63))
        rng_s4 = np.random.default_rng(rng.integers(1 << 63))

        frac_poiss_ind = simulate_blocking_given_interarrivals(
            inter_poiss_ind, m_servers,
            lambda R: exp_service(R, mean_service), rng_s3)

        frac_hyp_ind = simulate_blocking_given_interarrivals(
            inter_hyp_ind, m_servers,
            lambda R: exp_service(R, mean_service), rng_s4)

        diff_ind.append(frac_hyp_ind - frac_poiss_ind)

    diff_crn = np.array(diff_crn)
    diff_ind = np.array(diff_ind)

    def t_ci(arr):
        m  = arr.mean()
        se = arr.std(ddof=1) / np.sqrt(len(arr))
        hw = stats.t.ppf(0.975, len(arr)-1) * se
        return m, (m - hw, m + hw), se**2

    m_crn, ci_crn, var_crn = t_ci(diff_crn)
    m_ind, ci_ind, var_ind = t_ci(diff_ind)

    print("----- Difference  Δ = P_block(HyperExp) − P_block(Poisson) -----")
    print(f"Common random numbers:  Δ̂ = {m_crn:.5f},  95% CI = {ci_crn}")
    print(f"Independent streams :   Δ̂ = {m_ind:.5f},  95% CI = {ci_ind}")
    print(f"Variance ratio  Var_ind, Var_crn = {var_ind/var_crn:.2f}")

    return (diff_crn, diff_ind, (m_crn, ci_crn), (m_ind, ci_ind))

if __name__ == "__main__":
    compare_crn_vs_independent()


----- Difference  Δ = P_block(HyperExp) − P_block(Poisson) -----
Common random numbers:  Δ̂ = 0.01830,  95% CI = (0.013409692664540595, 0.02319030733545939)
Independent streams :   Δ̂ = 0.01737,  95% CI = (0.013093778284172599, 0.021646221715827408)
Variance ratio  Var_ind, Var_crn = 0.76


# 5.7

In [9]:
import numpy as np

def crude_mc_tail(a, n, rng):
    """Crude Monte-Carlo estimate and variance of P[Z>a]."""
    z = rng.standard_normal(n)
    w = (z > a).astype(float)
    return w.mean(), w.var(ddof=1) / n


def is_tail(a, n, sigma2, rng):
    """Importance-sampling estimate and variance with G ~ N(a, σ²)."""
    z = rng.normal(loc=a, scale=np.sqrt(sigma2), size=n)

    log_phi = -0.5 * z**2 - 0.5 * np.log(2 * np.pi)
    log_g   = -0.5 * (z - a)**2 / sigma2 - 0.5 * np.log(2 * np.pi * sigma2)
    w = np.exp(log_phi - log_g)

    g = (z > a) * w
    return g.mean(), g.var(ddof=1) / n

def gather_re(a_vals, n_vals, sigma2_list, seed=42):
    """Return a dict keyed by (a, σ², n) containing relative efficiencies."""
    rng_master = np.random.default_rng(seed)
    re = {}
    for a in a_vals:
        for s2 in sigma2_list:
            for n in n_vals:
                rng_cmc = np.random.default_rng(rng_master.integers(1 << 63))
                rng_is  = np.random.default_rng(rng_master.integers(1 << 63))
                _, var_cmc = crude_mc_tail(a, n, rng_cmc)
                _, var_is  = is_tail(a, n, s2, rng_is)
                re[(a, s2, n)] = var_cmc / var_is
    return re


if __name__ == "__main__":
    a_values      = (2, 4)
    n_values      = (10**3, 10**4, 10**5)
    sigma2_values = (0.5, 1.0, 2.0)

    re_dict = gather_re(a_values, n_values, sigma2_values)

    for a in a_values:
        print(f"\nRelative efficiency (RE) for a = {a}")
        print(f"{'n':>10} {'σ²':>6} {'RE':>12}")
        for n in n_values:
            for s2 in sigma2_values:
                re = re_dict[(a, s2, n)]
                print(f"{n:10d} {s2:6.1f} {re:12.2f}")



Relative efficiency (RE) for a = 2
         n     σ²           RE
      1000    0.5        19.38
      1000    1.0        16.31
      1000    2.0        12.84
     10000    0.5        27.91
     10000    1.0        19.32
     10000    2.0        10.85
    100000    0.5        28.51
    100000    1.0        18.52
    100000    2.0        11.81

Relative efficiency (RE) for a = 4
         n     σ²           RE
      1000    0.5         0.00
      1000    1.0         0.00
      1000    2.0         0.00
     10000    0.5         0.00
     10000    1.0         0.00
     10000    2.0         0.00
    100000    0.5      3362.77
    100000    1.0      8897.03
    100000    2.0      4407.96


# 5.8

In [10]:
import numpy as np, math
from scipy.optimize import minimize_scalar

def var_is(lmbda):
    num  = 1 - math.exp(-lmbda)
    EY2  = num / lmbda * (math.exp(lmbda + 2) - 1) / (lmbda + 2)
    return EY2 - (math.e - 1)**2         

res = minimize_scalar(var_is, bounds=(1e-4, 20), method='bounded')
lam_star = res.x                    
var_star = res.fun

rng = np.random.default_rng(42)
n   = 100_000
u   = rng.random(n)
x   = -np.log(1 - u * (1 - np.exp(-lam_star))) / lam_star   
weights = (1 - np.exp(-lam_star)) / lam_star * np.exp((lam_star + 1) * x)
mc_mean, mc_var = weights.mean(), weights.var(ddof=1)

print(lam_star, var_star, mc_mean, mc_var)


0.00010386511979656906 0.24208754306069125 1.7192487994000307 0.2418107893142285


# 5.9

In [13]:
import numpy as np

def sample_pareto(alpha, size, xm=1.0, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    u = rng.random(size)
    return xm / (1 - u) ** (1 / alpha)

def pareto_mean(alpha, xm=1.0):
    if alpha <= 1:
        raise ValueError("Mean is infinite when o ≤ 1")
    return alpha * xm / (alpha - 1)

def sample_size_biased_pareto(alpha, size, xm=1.0, rng=None):
    return sample_pareto(alpha - 1, size, xm, rng)

def is_estimator_mean(alpha, n=10_000, xm=1.0, seed=0):
    rng = np.random.default_rng(seed)
    x_g = sample_size_biased_pareto(alpha, n, xm, rng)
    mu = pareto_mean(alpha, xm)
    w   = mu / x_g
    y = x_g * w
    return y, y.var(ddof=1)

if __name__ == "__main__":
    α  = 2.5
    xm = 1.0
    n  = 1000

    y, var_emp = is_estimator_mean(α, n, xm, seed=42)
    print(f"First few estimates: {y[:5]}")
    print(f"Empirical variance over {n} samples: {var_emp:.3e}")
    print(f"True mean μ = {pareto_mean(α,xm):.4f}")


First few estimates: [1.66666667 1.66666667 1.66666667 1.66666667 1.66666667]
Empirical variance over 1000 samples: 5.735e-32
True mean μ = 1.6667
